<a href="https://colab.research.google.com/github/Rohanrathod7/my-ml-labs/blob/main/14_Model_Validation_in_Python/01_Basic_Modeling_in_scikit-learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 4. Dealing with Text Data


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import datetime as dt
# Import confusion matrix and train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import Ridge, Lasso, LogisticRegression, LinearRegression
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier




url = "https://raw.githubusercontent.com/Rohanrathod7/my-ml-labs/main/13_Feature_Engineering_for_Machine_Learning_in_Python/dataset/inaugural_speeches.csv"
# Read the CSV file

# Apply pd.to_numeric only to relevant columns, excluding 'text'
speech_df = pd.read_csv(url)
numeric_cols = ['Name', 'Inaugural Address', 'Date'] # Assuming these should be numeric if possible
for col in numeric_cols:
    speech_df[col] = pd.to_numeric(speech_df[col], errors='coerce')

display(speech_df.head())

,Name,Inaugural Address,Date,text
0,NaN,NaN,NaN,Fellow-Citizens of the Senate and of the House...
1,NaN,NaN,NaN,Fellow Citizens: I AM again called upon by th...
2,NaN,NaN,NaN,"WHEN it was first perceived, in early times, t..."
3,NaN,NaN,NaN,Friends and Fellow-Citizens: CALLED upon to u...
4,NaN,NaN,NaN,"PROCEEDING, fellow-citizens, to that qualifica..."


### Encoding Text





**Cleaning up your text**  
Unstructured text data cannot be directly used in most analyses. Multiple steps need to be taken to go from a long free form string to a set of numeric columns in the right format that can be ingested by a machine learning model. The first step of this process is to standardize the data and eliminate any characters that could cause problems later on in your analytic pipeline.

In this chapter you will be working with a new dataset containing the inaugural speeches of the presidents of the United States loaded as speech_df, with the speeches stored in the text column.

In [6]:
# Print the first 5 rows of the text column
print(speech_df["text"].head())

0    Fellow-Citizens of the Senate and of the House...
1    Fellow Citizens:  I AM again called upon by th...
2    WHEN it was first perceived, in early times, t...
3    Friends and Fellow-Citizens:  CALLED upon to u...
4    PROCEEDING, fellow-citizens, to that qualifica...
Name: text, dtype: object


In [7]:
# Replace all non letter characters with a whitespace
speech_df['text_clean'] = speech_df['text'].str.replace('[^a-zA-Z]', ' ', regex=True)

# Change to lower case
speech_df['text_clean'] = speech_df['text_clean'].str.lower()

# Print the first 5 rows of the text_clean column
print(speech_df['text_clean'].head())

# now your text strings have been standardized and cleaned up. You can now use this new column (text_clean) to extract information about the speeches.

0    fellow citizens of the senate and of the house...
1    fellow citizens   i am again called upon by th...
2    when it was first perceived  in early times  t...
3    friends and fellow citizens   called upon to u...
4    proceeding  fellow citizens  to that qualifica...
Name: text_clean, dtype: object


**High level text features**  
Once the text has been cleaned and standardized you can begin creating features from the data. The most fundamental information you can calculate about free form text is its size, such as its length and number of words. In this exercise (and the rest of this chapter), you will focus on the cleaned/transformed text column (text_clean) you created in the last exercise.

In [ ]:
# Find the length of each text
speech_df['char_cnt'] = speech_df['text_clean'].str.len()

# Count the number of words in each text
speech_df['word_cnt'] = speech_df['text_clean'].str.split().str.len()

# Find the average length of word
speech_df['avg_word_length'] = speech_df['char_cnt'] / speech_df['word_cnt']

# Print the first 5 rows of these columns
print(speech_df[['text_clean', 'char_cnt', 'word_cnt', 'avg_word_length']])

# These features may appear basic but can be quite useful in ML models.

### Word Count

**Counting words (I)**  
Once high level information has been recorded you can begin creating features based on the actual content of each text. One way to do this is to approach it in a similar way to how you worked with categorical variables in the earlier lessons.

For each unique word in the dataset a column is created.
For each entry, the number of times this word occurs is counted and the count value is entered into the respective column.
These "count" columns can then be used to train machine learning models.

In [9]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate CountVectorizer
cv = CountVectorizer()

# Fit the vectorizer
cv.fit(speech_df['text_clean'])

# Print feature names
print(cv.get_feature_names_out())

# this vectorizer can be applied to both the text it was trained on, and new texts.

['abandon' 'abandoned' 'abandonment' ... 'zealous' 'zealously' 'zone']


**Counting words (II)**  
Once the vectorizer has been fit to the data, it can be used to transform the text to an array representing the word counts. This array will have a row per block of text and a column for each of the features generated by the vectorizer that you observed in the last exercise.

The vectorizer to you fit in the last exercise (cv) is available in your workspace.

In [10]:
# Apply the vectorizer
cv_transformed = cv.transform(speech_df['text_clean'])

# Print the full array
cv_array = cv_transformed.toarray()

# Print the shape of cv_array
print(cv_array.shape)


# The speeches have 9043 unique words, which is a lot! In the next exercise, you will see how to create a limited set of features.

(58, 9043)


***Limiting your features***  
As you have seen, using the CountVectorizer with its default settings creates a feature for every single word in your corpus. This can create far too many features, often including ones that will provide very little analytical value.

For this purpose CountVectorizer has parameters that you can set to reduce the number of features:

min_df : Use only words that occur in more than this percentage of documents. This can be used to remove outlier words that will not generalize across texts.
max_df : Use only words that occur in less than this percentage of documents. This is useful to eliminate very common words that occur in every corpus without adding value such as "and" or "the".

In [12]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Specify arguments to limit the number of features generated
cv = CountVectorizer(min_df=0.2, max_df=0.8)

# Fit, transform, and convert into array
cv_transformed = cv.fit_transform(speech_df['text_clean'])
cv_array = cv_transformed.toarray()

# Print the array shape
print(cv_array.shape)

# Did you notice that the number of features (unique words) greatly reduced from 9043 to 818?

(58, 818)


**Text to DataFrame**  
Now that you have generated these count based features in an array you will need to reformat them so that they can be combined with the rest of the dataset. This can be achieved by converting the array into a pandas DataFrame, with the feature names you found earlier as the column names, and then concatenate it with the original DataFrame.

The numpy array (cv_array) and the vectorizer (cv) you fit in the last exercise are available in your workspace.

In [13]:
# Create a DataFrame with these features
cv_df = pd.DataFrame(cv_array,
                     columns=cv.get_feature_names_out()).add_prefix('Counts_')

# Add the new columns to the original DataFrame
speech_df_new = pd.concat([speech_df, cv_df], axis=1, sort=False)
print(speech_df_new.head())

# With the new features combined with the orginial DataFrame they can be now used for ML models or analysis.

   Name  Inaugural Address  Date  \
0   NaN                NaN   NaN   
1   NaN                NaN   NaN   
2   NaN                NaN   NaN   
3   NaN                NaN   NaN   
4   NaN                NaN   NaN   

                                                text  \
0  Fellow-Citizens of the Senate and of the House...   
1  Fellow Citizens:  I AM again called upon by th...   
2  WHEN it was first perceived, in early times, t...   
3  Friends and Fellow-Citizens:  CALLED upon to u...   
4  PROCEEDING, fellow-citizens, to that qualifica...   

                                          text_clean  Counts_abiding  \
0  fellow citizens of the senate and of the house...               0   
1  fellow citizens   i am again called upon by th...               0   
2  when it was first perceived  in early times  t...               0   
3  friends and fellow citizens   called upon to u...               0   
4  proceeding  fellow citizens  to that qualifica...               0   

   Counts_abi

### Term frequency-inverse document frequency

**Tf-idf**  
While counts of occurrences of words can be useful to build models, words that occur many times may skew the results undesirably. To limit these common words from overpowering your model a form of normalization can be used. In this lesson you will be using Term frequency-inverse document frequency (Tf-idf) as was discussed in the video. Tf-idf has the effect of reducing the value of common words, while increasing the weight of words that do not occur in many documents.

In [14]:
# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate TfidfVectorizer
# Instantiate TfidfVectorizer with max_features and stop_words
tv = TfidfVectorizer(max_features=100, stop_words='english')

# Fit the vectroizer and transform the data
tv_transformed = tv.fit_transform(speech_df['text_clean'])

# Create a DataFrame with these features
tv_df = pd.DataFrame(tv_transformed.toarray(),
                     columns=tv.get_feature_names_out()).add_prefix('TFIDF_')
print(tv_df.head())

# A job well done! Did you notice that counting the word occurences and calculating the Tf-idf weights are very similar?
# This is one of the reasons scikit-learn is very popular, a consistent API.

   TFIDF_action  TFIDF_administration  TFIDF_america  TFIDF_american  \
0      0.000000              0.133415       0.000000        0.105388   
1      0.000000              0.261016       0.266097        0.000000   
2      0.000000              0.092436       0.157058        0.073018   
3      0.000000              0.092693       0.000000        0.000000   
4      0.041334              0.039761       0.000000        0.031408   

   TFIDF_americans  TFIDF_believe  TFIDF_best  TFIDF_better  TFIDF_change  \
0              0.0       0.000000    0.000000      0.000000      0.000000   
1              0.0       0.000000    0.000000      0.000000      0.000000   
2              0.0       0.000000    0.026112      0.060460      0.000000   
3              0.0       0.090942    0.117831      0.045471      0.053335   
4              0.0       0.000000    0.067393      0.039011      0.091514   

   TFIDF_citizens  ...  TFIDF_things  TFIDF_time  TFIDF_today  TFIDF_union  \
0        0.229644  ...    

**Inspecting Tf-idf values**  
After creating Tf-idf features you will often want to understand what are the most highest scored words for each corpus. This can be achieved by isolating the row you want to examine and then sorting the the scores from high to low.

The DataFrame from the last exercise (tv_df) is available in your workspace.

In [15]:
# Isolate the row to be examined
sample_row = tv_df.iloc[0]

# Print the top 5 words of the sorted output
print(sample_row.sort_values(ascending=False).head())

# Do you think these scores make sense for the corresponding words?

TFIDF_government    0.367430
TFIDF_public        0.333237
TFIDF_present       0.315182
TFIDF_duty          0.238637
TFIDF_citizens      0.229644
Name: 0, dtype: float64


**Transforming unseen data**  
When creating vectors from text, any transformations that you perform before training a machine learning model, you also need to apply on the new unseen (test) data. To achieve this follow the same approach from the last chapter: fit the vectorizer only on the training data, and apply it to the test data.

For this exercise the speech_df DataFrame has been split in two:

- train_speech_df: The training set consisting of the first 45 speeches.
- test_speech_df: The test set consisting of the remaining speeches.

In [17]:
# Split data into training and testing sets
train_speech_df, test_speech_df = train_test_split(speech_df, test_size=0.2, random_state=42)

In [18]:
# Instantiate TfidfVectorizer
tv = TfidfVectorizer(max_features=100, stop_words='english')

# Fit the vectroizer and transform the data
tv_transformed = tv.fit_transform(train_speech_df['text_clean'])

# Transform test data
test_tv_transformed = tv.transform(test_speech_df['text_clean'])

# Create new features for the test set
test_tv_df = pd.DataFrame(test_tv_transformed.toarray(),
                          columns=tv.get_feature_names_out()).add_prefix('TFIDF_')
print(test_tv_df.head())

# Correct, the vectorizer should only be fit on the train set, never on your test set.

   TFIDF_administration  TFIDF_america  TFIDF_american  TFIDF_americans  \
0              0.130996       0.000000        0.106750         0.000000   
1              0.000000       0.000000        0.000000         0.000000   
2              0.071785       0.200906        0.058499         0.000000   
3              0.028327       0.000000        0.069252         0.000000   
4              0.000000       0.129432        0.200999         0.051832   

   TFIDF_believe  TFIDF_best  TFIDF_business  TFIDF_citizens  TFIDF_come  \
0       0.000000    0.000000        0.000000        0.234396    0.000000   
1       0.000000    0.194054        0.000000        0.054359    0.000000   
2       0.023369    0.081518        0.065955        0.034253    0.070106   
3       0.041496    0.036188        0.000000        0.101373    0.013832   
4       0.160586    0.070023        0.000000        0.029423    0.040147   

   TFIDF_commerce  ...  TFIDF_things  TFIDF_time  TFIDF_today  TFIDF_union  \
0        0.000

### N-grams

**Using longer n-grams**  
So far you have created features based on individual words in each of the texts. This can be quite powerful when used in a machine learning model but you may be concerned that by looking at words individually a lot of the context is being ignored. To deal with this when creating models you can use n-grams which are sequence of n words grouped together. For example:

- bigrams: Sequences of two consecutive words
- trigrams: Sequences of two consecutive words
These can be automatically created in your dataset by specifying the ngram_range argument as a tuple (n1, n2) where all n-grams in the n1 to n2 range are included.

In [20]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate a trigram vectorizer
cv_trigram_vec = CountVectorizer(max_features=100,
                                 stop_words='english',
                                 ngram_range=(3, 3))

cv_trigram = cv_trigram_vec.fit_transform(speech_df['text_clean'])

print(cv_trigram_vec.get_feature_names_out())

# Here you can see that by taking sequential word pairings, some context is preserved.

['ability preserve protect' 'agriculture commerce manufactures'
 'america celebrate th' 'america ideal freedom' 'america stands world'
 'america work preserve' 'american merchant marine'
 'americans fellow citizens' 'amity mutual concession' 'anchor peace home'
 'angel rides whirlwind' 'antirepublican tendencies preservation'
 'appear fellow citizens' 'ask bow heads' 'ask just government'
 'aspirations great people' 'assigned executive branch'
 'best ability preserve' 'best interests country' 'bless god bless'
 'bless united states' 'chief justice mr' 'children children children'
 'citizens united states' 'civil religious liberty' 'civil service reform'
 'commerce united states' 'confidence fellow citizens'
 'congress extraordinary session' 'constitution does expressly'
 'constitution united states' 'coordinate branches government'
 'day task people' 'defend constitution united'
 'distinguished guests fellow' 'does expressly say' 'equal exact justice'
 'era good feeling' 'executive bra

**Finding the most common words**  
Its always advisable once you have created your features to inspect them to ensure that they are as you would expect. This will allow you to catch errors early, and perhaps influence what further feature engineering you will need to do.

The vectorizer (cv) you fit in the last exercise and the sparse array consisting of word counts (cv_trigram) is available in your workspace.



In [22]:
# Create a DataFrame of the features
cv_tri_df = pd.DataFrame(cv_trigram.toarray(),
                         columns=cv_trigram_vec.get_feature_names_out()).add_prefix('Counts_')

# Print the top 5 words in the sorted output
print(cv_tri_df.sum().sort_values(ascending=False).head())

# that the most common trigram is constitution united states makes a lot of sense for US presidents speeches.

Counts_constitution united states    20
Counts_people united states          13
Counts_mr chief justice              10
Counts_preserve protect defend       10
Counts_president united states        8
dtype: int64
